In [1]:
import pandas as pd
import numpy as np

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Paths
RAW_DATA_PATH = "../data/raw/"

cases = pd.read_csv(RAW_DATA_PATH + "dataset_scin_cases.csv")
labels = pd.read_csv(RAW_DATA_PATH + "dataset_scin_labels.csv")
app_questions = pd.read_csv(RAW_DATA_PATH + "dataset_scin_app_questions.csv")
label_questions = pd.read_csv(RAW_DATA_PATH + "dataset_scin_label_questions.csv")

print("Cases shape:", cases.shape)
print("Labels shape:", labels.shape)
print("App Questions shape:", app_questions.shape)
print("Label Questions shape:", label_questions.shape)


Cases shape: (5033, 57)
Labels shape: (5033, 17)
App Questions shape: (11, 3)
Label Questions shape: (11, 4)


In [2]:
cases.columns


Index(['case_id', 'source', 'release', 'year', 'age_group', 'sex_at_birth', 'fitzpatrick_skin_type',
       'race_ethnicity_american_indian_or_alaska_native', 'race_ethnicity_asian',
       'race_ethnicity_black_or_african_american', 'race_ethnicity_hispanic_latino_or_spanish_origin',
       'race_ethnicity_middle_eastern_or_north_african', 'race_ethnicity_native_hawaiian_or_pacific_islander',
       'race_ethnicity_white', 'race_ethnicity_other_race', 'race_ethnicity_prefer_not_to_answer',
       'textures_raised_or_bumpy', 'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled',
       'body_parts_head_or_neck', 'body_parts_arm', 'body_parts_palm', 'body_parts_back_of_hand',
       'body_parts_torso_front', 'body_parts_torso_back', 'body_parts_genitalia_or_groin', 'body_parts_buttocks',
       'body_parts_leg', 'body_parts_foot_top_or_side', 'body_parts_foot_sole', 'body_parts_other',
       'condition_symptoms_bothersome_appearance', 'condition_symptoms_bleeding', 'condi

In [3]:
labels.columns


Index(['case_id', 'dermatologist_gradable_for_skin_condition_1', 'dermatologist_gradable_for_skin_condition_2',
       'dermatologist_gradable_for_skin_condition_3', 'dermatologist_skin_condition_on_label_name',
       'dermatologist_skin_condition_confidence', 'weighted_skin_condition_label',
       'dermatologist_gradable_for_fitzpatrick_skin_type_1', 'dermatologist_gradable_for_fitzpatrick_skin_type_2',
       'dermatologist_gradable_for_fitzpatrick_skin_type_3', 'dermatologist_fitzpatrick_skin_type_label_1',
       'dermatologist_fitzpatrick_skin_type_label_2', 'dermatologist_fitzpatrick_skin_type_label_3',
       'gradable_for_monk_skin_tone_india', 'gradable_for_monk_skin_tone_us', 'monk_skin_tone_label_india',
       'monk_skin_tone_label_us'],
      dtype='object')